# Use Case : Create cross DataBase static dataset for query Analysis with Stackoverflow Dataset

## Requirements  

- Use stack overflow dataset from [https://snap.stanford.edu/data/sx-stackoverflow.html](https://snap.stanford.edu/data/sx-stackoverflow.html)
- Convert the data to parquet format
- Create a external table with Hive meta store
- Demonstrate sample queries in SparkSQL, ThriftServer and Hive


------------------------------------------------------------------------------------------------------------------------

## Implementation 
- Explore teh data @ [https://archive.org/details/stackexchange](https://archive.org/details/stackexchange)
- Download the data from [https://archive.org/download/stackexchange](https://archive.org/download/stackexchange)



## Enable following commands to down load the dataset

In [1]:
# !mkdir -p  ~/ssp/data/downloads/stackoverflow/datascience/meta
# !mkdir -p  ~/ssp/data/downloads/stackoverflow/datascience/data
# !axel -n 4 https://archive.org/download/stackexchange/datascience.meta.stackexchange.com.7z -o ~/ssp/data/downloads/stackoverflow/datascience/meta/datascience.meta.stackexchange.com.7z
# !axel -n 4 https://archive.org/download/stackexchange/datascience.stackexchange.com.7z -o ~/ssp/data/downloads/stackoverflow/datascience/data/datascience.stackexchange.com.7z
# !wget https://ia600107.us.archive.org/27/items/stackexchange/readme.txt -O ~/ssp/data/downloads/stackoverflow/datascience/readme.txt

Move to the directory, remove old files and unzip new files

In [4]:
!cd ~/ssp/data/downloads/stackoverflow/datascience/meta && touch dummy.xml && rm *.xml && 7z x datascience.meta.stackexchange.com.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_IN,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz (906EA),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 672641 bytes (657 KiB)

Extracting archive: datascience.meta.stackexchange.com.7z
--
Path = datascience.meta.stackexchange.com.7z
Type = 7z
Physical Size = 672641
Headers Size = 293
Method = BZip2
Solid = +
Blocks = 1

    Everything is Ok

Files: 8
Size:       3964741
Compressed: 672641


In [5]:
!cd ~/ssp/data/downloads/stackoverflow/datascience/data  && touch dummy.xml && rm *.xml && 7z x datascience.stackexchange.com.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_IN,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz (906EA),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 48515842 bytes (47 MiB)

Extracting archive: datascience.stackexchange.com.7z
--
Path = datascience.stackexchange.com.7z
Type = 7z
Physical Size = 48515842
Headers Size = 326
Method = BZip2
Solid = +
Blocks = 4

      6% 1 - Comments.xm                     11% 2 - PostHistory.x                       17% 2 - PostHistory.x                       23% 2 - PostHistory.x                       27% 2 - PostHistory.x                       32% 2 - PostHistory.x                       38% 2 - PostHistory.x                       44% 2 - PostHistory.x                       50% 2 - PostHistory.x                       55% 4 - Posts.x                 61% 4 - Posts.x                 66% 4 - Posts.x                 72% 4 - Posts.x                 77%

## List down the files in `meta` and `data` folders

In [6]:
!ls ~/ssp/data/downloads/stackoverflow/datascience/data

Badges.xml			  PostHistory.xml  Tags.xml
Comments.xml			  PostLinks.xml    Users.xml
datascience.stackexchange.com.7z  Posts.xml	   Votes.xml


In [7]:
!ls ~/ssp/data/downloads/stackoverflow/datascience/meta

Badges.xml			       PostHistory.xml	Tags.xml
Comments.xml			       PostLinks.xml	Users.xml
datascience.meta.stackexchange.com.7z  Posts.xml	Votes.xml


## Dataset XML Description

In [8]:
!cat ~/ssp/data/downloads/stackoverflow/datascience/readme.txt

﻿ - Format: 7zipped
 - Files:
   - **badges**.xml
       - UserId, e.g.: "420"
       - Name, e.g.: "Teacher"
       - Date, e.g.: "2008-09-15T08:55:03.923"
   - **comments**.xml
       - Id
       - PostId
       - Score
       - Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
       - CreationDate, e.g.:"2008-09-06T08:07:10.730"
       - UserId
   - **posts**.xml
       - Id
       - PostTypeId
          - 1: Question
          - 2: Answer
       - ParentID (only present if PostTypeId is 2)
       - AcceptedAnswerId (only present if PostTypeId is 1)
       - CreationDate
       - Score
       - ViewCount
       - Body
       - OwnerUserId
       - LastEditorUserId
       - LastEditorDisplayName="Jeff Atwood"
       - LastEditDate="2009-03-05T22:28:34.823"
       - LastActivityDate="2009-03-11T12:51:01.480"
       - CommunityOwnedDate="2009-03-11T12:51:01.480"
       - ClosedDate="2009-03-11T12:51:01.480"
       - Title=
       - Tags=
       - AnswerCount
       -

In [9]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnull, desc
import os

In [10]:
!ls ../../libs/spark-xml_2.11-0.9.0.jar

../../libs/spark-xml_2.11-0.9.0.jar


In [11]:
spark_master = "local[4]"
# spark_master = "spark://IMCHLT276:7077"

In [12]:
spark = SparkSession.builder. \
                appName("stakoverflow_dataset"). \
                config("spark.jars", "../../libs/spark-xml_2.11-0.9.0.jar"). \
                config("spark.executor.memory", "3g"). \
                config("spark.executor.cores", 3). \
                config("spark.cores.max", 6). \
                master(spark_master). \
                enableHiveSupport(). \
                getOrCreate()

In [13]:
spark

In [14]:
!ls ~/ssp/data/downloads/stackoverflow/datascience/data

Badges.xml			  PostHistory.xml  Tags.xml
Comments.xml			  PostLinks.xml    Users.xml
datascience.stackexchange.com.7z  Posts.xml	   Votes.xml


In [15]:
def load_data(path, debug=True):
    """
    Loads all the Stackoverflow files as PySpark DataFrame. Column names are prefixed with `_` by default.
    :path : Root directory of the zip extracted folder
    returns: Dictionary of DataFrames where each key(lowercase) represents the respective XML file
    """
    files = ["Badges.xml", "PostHistory.xml", "Tags.xml", "Comments.xml", "PostLinks.xml", "Users.xml", "Posts.xml", "Votes.xml"]
    dfs = {}
    for key in files:
        full_path = path + key
        print(f"Loading file {full_path}")
        key_ = key.lower().replace(".xml", "")
        dfs[key_] = spark.read.format("com.databricks.spark.xml").option("rootTag", "Tags").option("attributePrefix", "_").option("rowTag", "row").load(full_path)
        if debug:
            print(f"File : {key}")
            dfs[key_].show(5)
    return dfs

path = "file:///" + os.path.expanduser("~/ssp/data/downloads/stackoverflow/datascience/data/")
dfs = load_data(path=path) 

Loading file file:////home/mageswarand/ssp/data/downloads/stackoverflow/datascience/data/Badges.xml
File : Badges.xml
+------+--------------------+---+--------------+---------+-------+
|_Class|               _Date|_Id|         _Name|_TagBased|_UserId|
+------+--------------------+---+--------------+---------+-------+
|     3|2014-05-13T23:06:...|  1|      Informed|    false|      1|
|     3|2014-05-13T23:11:...|  2|Autobiographer|    false|      2|
|     3|2014-05-13T23:20:...|  3|Autobiographer|    false|      4|
|     3|2014-05-13T23:20:...|  4|Autobiographer|    false|      5|
|     3|2014-05-13T23:20:...|  5|Autobiographer|    false|      8|
+------+--------------------+---+--------------+---------+-------+
only showing top 5 rows

Loading file file:////home/mageswarand/ssp/data/downloads/stackoverflow/datascience/data/PostHistory.xml
File : PostHistory.xml
+--------+--------------------+---+------------------+-------+--------------------+--------------------+----------------+-----

**Find top 5 Posts with more answers and find what those questions are?**


In [16]:
questions = dfs["posts"].where(~isnull(col("_AnswerCount"))).sort(desc("_AnswerCount")).select(["_AnswerCount", "_Body"]).collect()[:5]

In [17]:
for q in questions:
    print(q["_Body"])
    print("-------------------------------------------")

<p>One of the common problems in data science is gathering data from various sources in a somehow cleaned (semi-structured) format and combining metrics from various sources for making a higher level analysis. Looking at the other people's effort, especially other questions on this site, it appears that many people in this field are doing somewhat repetitive work. For example analyzing tweets, facebook posts, Wikipedia articles etc. is a part of a lot of big data problems.</p>

<p>Some of these data sets are accessible using public APIs provided by the provider site, but usually, some valuable information or metrics are missing from these APIs and everyone has to do the same analyses again and again. For example, although clustering users may depend on different use cases and selection of features, but having a base clustering of Twitter/Facebook users can be useful in many Big Data applications, which is neither provided by the API nor available publicly in independent data sets.</p>


# Stay tuned for more updates!

https://towardsdatascience.com/getting-started-with-data-analytics-using-jupyter-notebooks-pyspark-and-docker-57c1aaab2408

https://lgoerl.github.io/spark-overflow-analysis.html